In [1]:
import requests
import bs4
import re

def filterCountry(tag):
    if tag is None:
        return False
    q = re.compile("country_of_origin")
    m = q.search(tag)
    if m:
        return True
    return False

def findCountry(key):
    res=requests.get('http://www.imdb.com/title/' + key + '/')
    res.raise_for_status()
    soup=bs4.BeautifulSoup(res.text,"lxml")
    result=soup.select('#titleDetails')
    if result:
        c = result[0].find_all(href=filterCountry)
        strs = result[0].stripped_strings
        grab = False
        date = None
        for str in strs:
            if grab:
                date = str
                break

            if("Release Date:" == str):
                grab = True
        if c==None:
            return None,date
        else:
            return c[0].getText(), date


In [2]:
class Test(object):
    def __init__(self,data):
         self.__dict__=json.loads(data)
    def myprint(self):
         for key,value in self.__dict__.items():
            print(key,value)
    def get(self,key):
        if key in self.__dict__.keys():
            return self.__dict__[key]
        return None
    def put(self, key, value):
        self.__dict__[key] = value
    def toJSON(self):
        return json.dumps(self.__dict__)

In [17]:
import json
import os

def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

with open('IMDB.json','r', encoding='utf8')as f:
    movies=[Test(line) for line in f]
print("run")

with open('output_partai_1.json', 'w', 1) as f:   
    for movie in movies:
        country = movie.get('country')
        year = movie.get("year")
        if(((country is None) or ('fficial' in country)) and (year >= 1960)):
            key = movie.get('imdb')
            try:
                country, date = findCountry(key)
            except requests.exceptions.RequestException as e:  # This is the correct syntax
                print (e)
                break
            except Exception as e:
                print(e)
                break
            
            if date:
                coun = find_between(date,'(',')')
                #print(coun)
            
            if coun == 'USA':
                if(country is not None):
                    movie.put('country', country)
                    #print(country)
                if(date is not None):
                    movie.put('releasedate', date)
  
                js = movie.toJSON()
                f.write(js + '\n')
print("All done")

run
All done
